In [3]:
import pandas as pd
import numpy as np
import copy
import itertools
# Package for Levenstein string distance
import jellyfish
import time

In [4]:
path = "C://Users//Dimitri//Desktop//ENSAE3A//NetworkData//Data//"

In [5]:
attrs = pd.read_csv(path + "attrs_sub.csv")

In [7]:
def authors_parser(authors_string, sep=";"):
    return authors_string.split("; ")

In [13]:
attrs["authors_list"] = attrs["authors"].apply(authors_parser)

In [8]:
def normalized_edit_distance(str1, str2):
    dist = jellyfish.levenshtein_distance(str1, str2)
    return dist / max(len(str1), len(str2))

In [9]:
def parse_names(author):
    if "," in author:
        splitted = author.split(", ")
        lastname = splitted[0]
        if len(splitted) == 1:
            firstname = ""
        else : 
            firstname = splitted[1]
            if firstname[0] == " ":
                firstname = firstname[1:]
    else:
        splitted = author.split(" ")
        if len(splitted) > 1:
            lastname = ""
            for part in splitted[1:]:
                lastname += " " + part
            firstname = splitted[0]
        else:
            firstname = ""
            lastname = splitted[0]
    return firstname, lastname

In [10]:
def sortby_firstname_len(author):
    firstname, lastname = parse_names(author)
    return len(firstname)
   

In [14]:
# Fill all authors
authors = []
for author in attrs["authors_list"]:
    authors += author
# Remove duplicates
authors = list(set(authors))

In [15]:
# Sortby first name length
authors_red = authors[0:1000]
start = time.clock()
#authors.sort(key=sortby_firstname_len, reverse=True)
authors_red.sort(key=sortby_firstname_len, reverse=True)
end = time.clock()
print(end - start)

0.0016909460402611087


In [16]:
authors

['Kwiek, Maksymilian',
 'George E. Barnett',
 'Dijk, Dick Van',
 'Spychalski, John C',
 'Matthew Serfling',
 'Litov, Lubomir',
 'Skrzypacz, Andrzej',
 'Paolo Pin',
 'Ashley, Richard A',
 'Hu, Wei-Yin',
 'James W. Albrecht',
 'Vicente Cunat',
 'Babus, Ana',
 'Madrian, Brigitte C',
 'Fitzgerald, Doireann',
 'Dungey, Mardi',
 'Saidenberg, Marc R.',
 'Vodopivec, Milan',
 'Attiat F. Ott',
 'Kilcollin, T. Eric',
 'Raman, Kartik',
 'Minardi, Stefania',
 'Clifton H. Keeps Jr.',
 'Jaroslav Borovička',
 'James R. Hines Jr.',
 'Charles R. Link',
 'Raymond Gradus',
 'Deza, Monica',
 'Roman Kozhan',
 'Dorsey Richardson',
 'Tarsalewska, Monika',
 'Joseph Gyourko',
 'I. L. Sharfman',
 'Lorenzo Codogno',
 'J. Douglas Brown',
 'Prüfer, Patricia',
 'G. M. Youssef',
 'Sjoquist, David L',
 ' Marshall, David A',
 'Schröder, Marcel',
 'Walker, Francis E.',
 'Henderson, J V',
 'Caroline Paunov',
 'Sean Mulholland',
 'Spiegel, M.',
 'Marie-Claire Villeval',
 'Östberg, Per',
 'Jeffrey Liebner',
 'Tomohiro Hira

In [181]:
start = time.clock()
thresh = 0.25
authors_frame = pd.DataFrame(columns=["first_name", "last_name", "similar"])
for author in authors:
    firstname, lastname = parse_names(author)
    search = authors_frame["last_name"].str.contains(lastname)
    ind_search = search[search == True].index
    exists_similar = False
    for ind in ind_search :
        dist = normalized_edit_distance(authors_frame["first_name"][ind], 
                                        firstname)
        dist_test = dist <= thresh
        if dist_test :
            exists_similar = True
            authors_frame["similar"][ind].append(author)    
        first_letter_test1 = (len(firstname) == 1) and (firstname[0] == authors_frame["first_name"][ind][0])
        if first_letter_test1:
            exists_similar = True
            authors_frame["similar"][ind].append(author)     
    if not exists_similar:
        df = pd.DataFrame(columns=["first_name", "last_name", "similar"], index=[0])
        df.set_value(0, "first_name", firstname)
        df.set_value(0, "last_name", lastname)
        df.set_value(0, "similar", [author])
        authors_frame = authors_frame.append(df, ignore_index=True)
end = time.clock()
print(end - start)

C:\Users\Dimitri\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


KeyboardInterrupt: 

In [32]:
df_authors = pd.DataFrame(authors, columns=["original"])

In [189]:
df_authors

,original
0,"Lopes, Francesco Saverio Stentella"
1,"Hidalgo, Fernando Perez de Gracia"
2,"Werlang, Sergio Ribeiro da Costa"
3,"Pería, María Soledad Martínez"
4,"Pería, Maria Soledad Martínez"
5,"Knarvik, Karen Helene Midelfart"
6,"Paul, Catherine J. Morrison"
7,"Ramamurtie, Buddhavarapu Sailesh"
8,"Aïssa, Mohamed Safouane Ben"
9,"Jeanneney, Sylviane Guillaumont"


In [33]:
def uniformize_names(str1):
    if "," in str1:
        splitted = str1.split(", ")
        if len(splitted) > 1:
            str1bis = splitted[1] + " " + splitted[0]
        else :
            str1bis = str1
        return str1bis
    else:
        return str1

In [34]:
df_authors["uniformat"] = df_authors["original"].apply(uniformize_names)

In [35]:
df_authors_reduced = df_authors.loc[0:1000, :]

In [36]:
df_authors_reduced

,original,uniformat
0,"Kwiek, Maksymilian",Maksymilian Kwiek
1,George E. Barnett,George E. Barnett
2,"Dijk, Dick Van",Dick Van Dijk
3,"Spychalski, John C",John C Spychalski
4,Matthew Serfling,Matthew Serfling
5,"Litov, Lubomir",Lubomir Litov
6,"Skrzypacz, Andrzej",Andrzej Skrzypacz
7,Paolo Pin,Paolo Pin
8,"Ashley, Richard A",Richard A Ashley
9,"Hu, Wei-Yin",Wei-Yin Hu


In [21]:
def edit_apply(str1, str2):
    return normalized_edit_distance(str1, str2)

In [22]:
def edit_score_tab(auth, auth_df):
    tab = pd.DataFrame(data=auth_df["uniformat"].copy(), columns=["uniformat"])
    tab["edit"] = tab["uniformat"].apply(lambda x: edit_apply(auth, x))
    return tab

In [46]:
df_authors_reduced = df_authors.iloc[0:1000, :].copy()

In [251]:
def map_authors(auths_df, thresh):
    auths_df["equivalent"] = None
    for i in auths_df.index:
        auth = auths_df.get_value(i, "uniformat")
        auth_original = auths_df.get_value(i, "original")
        auths_df.set_value(i, "equivalent", [auth_original])
        search = auths_df.drop(i, axis=0)["uniformat"].str.contains(auth.split(" ")[-1])
        ind_search = search[search == True].index
        if len(ind_search) >= 1:
            sub_auths_df = auths_df.iloc[ind_search, :]
            sub_auths_df["edit"] = sub_auths_df["uniformat"].apply(lambda x: edit_apply(auth, x))
            sub_inds = sub_auths_df[sub_auths_df["edit"] <= thresh].index
            if len(sub_inds) > 0:
                for j in sub_inds:
                    auths_df.loc[i, "equivalent"].append(auths_df.get_value(j, "original"))
    return auths_df

In [72]:
def map_authors(auths_df, thresh):
    cleaned_df = pd.DataFrame(columns=["original", "uniformat", "equivalent"])
    perfs = []
    for i in auths_df.index:
        author = auths_df.loc[i, "uniformat"]
        author_original = auths_df.loc[i, "original"]
        splitted = author.split(" ")[-1]
        last = splitted[-1]
        search = cleaned_df["uniformat"].str.contains(last)
        ind_search = search[search == True].index
        first = splitted[0]
        prop = (len(first) == 1) or ((len(first) == 2) and ("." in first))
        exists_similar = False
        for ind in ind_search :
            dist = normalized_edit_distance(cleaned_df["uniformat"][ind], 
                                            author)
            perfs.append(end - start)
            dist_test = dist <= thresh
            if dist_test :
                exists_similar = True
                cleaned_df["equivalent"][ind].append(author_original)    
            # first_letter_test1 = prop and (first[0] == cleaned_df["uniformat"][ind][0])
            # if first_letter_test1:
                # exists_similar = True
                # cleaned_df["equivalent"][ind].append(author_original) 
            perfs.append(end - start)
        if not exists_similar:
            df = pd.DataFrame(columns=["original", "uniformat", "equivalent"], index=[0])
            df.set_value(0, "original", author_original)
            df.set_value(0, "uniformat", author)
            df.set_value(0, "equivalent", [author_original])
            cleaned_df = cleaned_df.append(df, ignore_index=True)
    return cleaned_df, perfs

In [73]:
start = time.clock()
cleaned, perfs = map_authors(df_authors_reduced, 0.1)
end = time.clock()
print(end - start)

51.836458917794516


In [54]:
import matplotlib.pyplot as plt

In [74]:
sum(perfs)

94.86415040446445

In [61]:
df_authors_reduced.iloc[688, :]

original      Mian, Shehzad L
uniformat     Shehzad L  Mian
Name: 688, dtype: object

In [75]:
inds = []
for b in cleaned.iterrows():
    if len(b[1]["equivalent"]) > 1:
        inds.append(b[0])

In [76]:
inds

[37, 150, 668]

In [81]:
cleaned.loc[668, "equivalent"]

['Tille, Cedric', 'Tille, Cédric']

In [ ]:
jellyfish.levenhstein